In [1]:
import pandas as pd
import os

In [2]:
## MAIN FUNCTION - input

## INPUT FILE
# codemaps = pd.read_csv('codemaps.csv')
country_names = pd.read_csv('../../../data/dl1_countrycodeorg_country_name.csv')
UNcode_names = pd.read_csv('WPP2019_F01_LOCATIONS_DB.csv')
UNcode_names = UNcode_names[['Country Code', 'LocID']]

df_input_pop = pd.read_csv('WPP2019_PopulationByAgeSex_Medium.csv')
df_input_pop_total = pd.read_csv('WPP2019_TotalPopulationBySex.csv')

start_year = 2010
end_year = 2051
## OUTPUT FILE
save_file_name = "population_un.csv"
save_file_name_total = "population_total.csv"

In [3]:
## Merge data

# ages = {'0-4': 'd0', '5-9': 'd05', '10-14': 'd10', '15-19': 'd15', 
#         '20-24': 'd20', '25-29': 'd25', '30-34': 'd30', '35-39': 'd35', 
#         '40-44': 'd40', '45-49': 'd45', '50-54': 'd50', '55-59': 'd55', 
#         '60-64': 'd60', '65-69': 'd65', '70-74': 'd70', '75-79': 'd75', 
#         '80-84': 'd80', '85-89': 'd85', '90-94': 'd90', '95-99': 'd95', 
#         '100+': 'd95'}
ages = {'0-4': 'd0', '5-9': 'd5', '10-14': 'd10', '15-19': 'd15', 
        '20-24': 'd20', '25-29': 'd25', '30-34': 'd30', '35-39': 'd35', 
        '40-44': 'd40', '45-49': 'd45', '50-54': 'd50', '55-59': 'd55', 
        '60-64': 'd60', '65-69': 'd65', '70-74': 'd65', '75-79': 'd65', 
        '80-84': 'd65', '85-89': 'd65', '90-94': 'd65', '95-99': 'd65', 
        '100+': 'd65'}

# def changevalue(x):
#     return x * 1000

In [4]:
print('Please calculate the numbers and verify!')
len(df_input_pop), len(df_input_pop['LocID'].unique()), len(df_input_pop['Time'].unique()), len(df_input_pop['AgeGrp'].unique())

Please calculate the numbers and verify!


(1404753, 443, 151, 21)

In [5]:
df = df_input_pop[['LocID', 'Location', 'Variant', 'Time', 'AgeGrp', 'PopMale', 'PopFemale']]
# df = df[df['Variant']=='Medium']
df = df.rename(columns={"Time": "year", "AgeGrp": 'age'})
df['age'] = df['age'].apply(lambda x: ages[x])

df_male = df[['LocID', 'Location', 'Variant', 'year', 'age', 'PopMale']]
df_male['sex'] = 'M'
df_male = df_male.rename(columns={"PopMale": "population"})

df_female = df[['LocID', 'Location', 'Variant', 'year', 'age', 'PopFemale']]
df_female['sex'] = 'F'
df_female = df_female.rename(columns={"PopFemale": "population"})

df_combined = pd.concat([df_male, df_female])

gb = df_combined.groupby(['LocID', 'sex', 'age', 'year']).sum()
print(len(gb))
gb = gb.reset_index()

1873004


In [6]:
dff = pd.pivot_table(gb, columns=['year'], index=['LocID', 'sex', 'age'], values='population')
dff = dff.reset_index()

In [7]:
pop = dff.merge(UNcode_names, on='LocID').drop(columns=['LocID'])
pop = pop.dropna(subset=['Country Code'])
pop.replace('\s+', '', regex=True, inplace=True)
pop = pop[pop['Country Code'] != '']
pop = pop.set_index(['Country Code', 'sex', 'age'])
pop = (pop * 1000).astype('int')
pop.reset_index().to_csv(save_file_name, index=False)

In [8]:
import pandas as pd
def check_countries(filename):
    df = pd.read_csv(filename)
    countries = df['Country Code'].unique()
    country_names = pd.read_csv('../../../data/dl1_countrycodeorg_country_name.csv')
    GBD_countries = country_names[country_names["country"].notnull()]['Country Code']
    WB_countries = country_names[country_names["WBCountry"].notnull()]['Country Code']
    plus_GBD = sorted(list(set(countries) - set(GBD_countries)))
    sub_GBD = sorted(list(set(GBD_countries) - set(countries)))
    plus_WB = sorted(list(set(countries) - set(WB_countries)))
    sub_WB = sorted(list(set(WB_countries) - set(countries)))

    print ('Subtraction from WB:', sub_WB, len(sub_WB))
    print ('Plus from WB:', plus_WB, len(plus_WB))
    print ('Subtraction from GBD :', sub_GBD, len(sub_GBD))
    print ('Plus from GBD :', plus_GBD, len(plus_GBD))
check_countries(save_file_name)

Subtraction from WB: ['AND', 'ASM', 'BMU', 'CYM', 'DMA', 'FRO', 'GIB', 'GRL', 'IMN', 'KNA', 'LIE', 'MAF', 'MCO', 'MHL', 'MNP', 'NRU', 'PLW', 'SMR', 'SXM', 'TCA', 'TUV', 'VGB', 'XKX'] 23
Plus from WB: ['ESH', 'GLP', 'GUF', 'MTQ', 'MYT', 'PSE', 'REU', 'TWN'] 8
Subtraction from GBD : ['AND', 'ASM', 'BMU', 'COK', 'DMA', 'GRL', 'KNA', 'MCO', 'MHL', 'MNP', 'NIU', 'NRU', 'PLW', 'SMR', 'TKL', 'TUV'] 16
Plus from GBD : ['ABW', 'CHI', 'CUW', 'ESH', 'GLP', 'GUF', 'HKG', 'MAC', 'MTQ', 'MYT', 'NCL', 'PYF', 'REU'] 13


## Population Total

### We do not sum up all population by sex and age. All countries have total population.

In [9]:
print('Please calculate the numbers and verify!')
df_tmp = df_input_pop_total[df_input_pop_total['Variant']=='Medium']
print(len(df_tmp), len(df_tmp['Variant'].unique()), len(df_tmp['LocID'].unique()), len(df_tmp['Time'].unique()))


Please calculate the numbers and verify!
72027 1 477 151


In [10]:
pop_total = df_tmp.merge(UNcode_names, on='LocID')
pop_total = pop_total[['Country Code', 'Time', 'PopTotal']]
pop_total = pop_total.dropna(subset=['Country Code'])
pop_total.replace('\s+', '', regex=True, inplace=True)
pop_total = pop_total[pop_total['Country Code'] != '']
pop_total = pd.pivot_table(pop_total, columns=['Time'], index=['Country Code'], values='PopTotal')
pop_total = (pop_total * 1000).astype('int')
pop_total.reset_index().to_csv(save_file_name_total, index=False)

In [11]:
check_countries(save_file_name_total)

Subtraction from WB: ['XKX'] 1
Plus from WB: ['AIA', 'BES', 'BLM', 'COK', 'ESH', 'FLK', 'GLP', 'GUF', 'MSR', 'MTQ', 'MYT', 'NIU', 'PSE', 'REU', 'SHN', 'SPM', 'TKL', 'TWN', 'VAT', 'WLF'] 20
Subtraction from GBD : [] 0
Plus from GBD : ['ABW', 'AIA', 'BES', 'BLM', 'CHI', 'CUW', 'CYM', 'ESH', 'FLK', 'FRO', 'GIB', 'GLP', 'GUF', 'HKG', 'IMN', 'LIE', 'MAC', 'MAF', 'MSR', 'MTQ', 'MYT', 'NCL', 'PYF', 'REU', 'SHN', 'SPM', 'SXM', 'TCA', 'VAT', 'VGB', 'WLF'] 31


## Population in GBD

In [54]:
## MAIN FUNCTION - input
## INPUT FILE
def find_csv_filenames(path_to_dir, suffix=".CSV" ):
    filenames = os.listdir(path_to_dir)
    return [filename for filename in filenames if filename.endswith(suffix)]

def change_df_columnnames(df): # change columnnames if download data with id and names
    new_col = [col.split("_")[0] if col.split("_")[-1] == "name" else col for col in df.columns]
    df.columns = new_col
    del_col = [col for col in df.columns if (col.split("_")[-1] == "id") and col.split("_")[0] in new_col]
    df_new = df.drop(columns=del_col)
    df_new.rename(columns={'year_id' : "year"}, inplace=True)
    return df_new

file_dir = './IHME_GBD_2019_POP_2010_2019_0/'
csv_files = find_csv_filenames(file_dir)
## time
MAXSIZE = 1



ages_to_use = ['Under 5', '5 to 9', '10 to 14', '15 to 19', '20 to 24',
       '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49',
       '50 to 54', '55 to 59', '60 to 64', '65 to 69', '70 to 74',
       '75 plus']
ages = {'Under 5': 'd0', '5 to 9': 'd5', '10 to 14': 'd10', '15 to 19': 'd15', 
        '20 to 24': 'd20', '25 to 29': 'd25', '30 to 34': 'd30', '35 to 39': 'd35', 
        '40 to 44': 'd40', '45 to 49': 'd45', '50 to 54': 'd50', '55 to 59': 'd55', 
        '60 to 64': 'd60', '65 to 69': 'd65', '70 to 74': 'd65', '75 plus': 'd65'}

pieces = []
for file in csv_files:
    pop_input = pd.read_csv(file_dir + file)
    pop_input = change_df_columnnames(pop_input)
    pop_input = pop_input[pop_input['age'].isin(ages_to_use)]
    pop_input = pop_input[pop_input['year']>=2010]
    pieces.append(pop_input)
    print(len(pop_input))
pop_input = pd.concat(pieces)

32400
32400
32400
32400
32400
32400
32400
32400
32400
32400


In [76]:
countries_pri = pd.read_csv('../../data/dl1_countrycodeorg_country_name.csv')
print(len(pop_input))
pop_gbd = pop_input
pop_gbd['sex'].replace(to_replace='female', value='F', inplace=True)
pop_gbd['sex'].replace(to_replace='male', value='M', inplace=True)
pop_gbd = pop_gbd[pop_gbd['sex'] != 'both']
pop_gbd['age'] = pop_gbd['age'].apply(lambda x: ages[x])
print(len(pop_gbd)) ##
print('Please calculate the numbers and verify!')
gb = pop_gbd.groupby(['measure', 'location', 'sex', 'age', 'metric', 'year']).sum()
print(len(gb))
pop_gbd = gb.reset_index()
print(len(pop_gbd['location'].unique()), len(df['sex'].unique()), len(df['age'].unique()),len(df['year'].unique()))

324000
216000
Please calculate the numbers and verify!
188160
672 2 14 10


/Users/caozhong/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [77]:
countries = pd.read_csv('../../data/dl1_countrycodeorg_country_name.csv')
code_map = dict(zip(countries.country, countries['Country Code'])) 
pop_gbd = pop_gbd[pop_gbd['location'].isin(countries.country)]
pop_gbd['Country Code'] = pop_gbd['location'].apply(lambda x:code_map[x])

In [78]:
pop_gbd_saved = pd.pivot_table(pop_gbd, columns=['year'], index=['Country Code', 'sex', 'age'], values='val')
pop_gbd_saved.reset_index().to_csv('population_gbd.csv',index=False)